# Instruction Finetuning LLMs with QLoRA

The process in this notebook follows from: https://www.philschmid.de/instruction-tune-llama-2

In this lab, we will demonstrate finetuning a Large Language Model to better respond to instructions.  This is frequently referred to as Instruction Fine-Tuning. 

- [Preparing the Dataset](#preparing-the-dataset)
- [Selecting the Base Pre-trained Model](#selecting-the-base-pre-trained-model)
- [Finetuning the Model](#finetuning-the-model)

## Preparing the Dataset

Fine-tuning LLMs is primarily used for teaching the model new behavior, such as better responding to instructions, responding with certain tones, or acting more as a conversational chatbot.  

The dataset for finetuning LLMs are text entries formatted in the way ***THAT WE WISH FOR AN INTERACTION WITH THE MODEL TO LOOK LIKE***.  For example, if we wish for the model to follow instructions better, we should provide a dataset which gives examples of it following instructions.  **This is almost exactly like few-shot prompting, but reinforcing the behavior even further by actually changing the weights of the model.**  After fine-tuning, few-shot prompts will not be necessary.


### Dataset using `datasets`

The dataset that we will be using for instruction fine-tuning is a dataset hand-curated by databricks for instruction following called "dolly-15k".  Quality data is key for getting finetuning to work successfully.

In [ ]:
from datasets import load_dataset

# Download the dataset from HuggingFace
dataset = load_dataset("databricks/databricks-dolly-15k", split = "train")
# dataset = dataset.select(range(100)) # limit the size for testing
dataset

The base dataset contains columns for an `instruction`, an optional `context`, and an `response` that we want the bot to respond to.  However, to feed it into the model for finetuning, we need to combine each column so that 1 row corresponds to 1 example interaction with the model.  

This 1 entry should be an example to the LLM about:

1. How we wish to interact with the model (prompt)
2. How we want the model to respond

Remember, these generative LLMs are trained to read in a provided prompt, and essentially auto-complete the text!

In [ ]:
def format_instruction(sample):
    # This function takes a row of the above dataset and returns a single text string
    # with an example interaction

	return f"""### Instruction:
    Use the following Input and come up with a structured response.
	
    ### Input:
    {sample['instruction']}

    ### Response:
    {sample['response']}
    """

For the purposes of instruction following, the example interaction from the above function will look like:

```
### Instruction:
Use the following Input and come up with a structured response.

### Input:
{instruction}

### Response:
{response}
```

At inference time, we will pass in the top part of our prompt, with our instruction,
```
### Instruction:
Use the following Input and come up with a structured response.

### Input:
{instruction}

### Response:
```
and let the model autocomplete to fill in the `{response}`.

## Selecting the Base Pre-trained Model

Once we have the data, we can select the base model that we would like to fine tune for this behavior.  

The model that we will select is the `Llama-2-7b` base model.  This is a 7b parameter model, quite small in the grand scheme of LLMs, but one that produces good quality results, especially compared to models of similar sizes.  Note that we fine-tune the base version of the model, not the chat fine-tuned model.  For introducing new behavior, it is generally better to fine tune the base version of the model rather than one that is already fine-tuned for another task.

### Quantization using `bitsandbytes`

LLMs are extremely memory intensive.  One trick that is commonly used when working with LLMs to reduce memory usage as well as increase computational speed for both inference and training, is reducing the precision of the weights from full precision 32-bit floating points (fp32) to lower precisions such as int8, fp4, nf4, etc.  This is known as quantization.  Research has shown that quantization often times has minimal impact on the quality of generations, but this is on a case-by-case basis. 

In this example, we will be quantizing and fine-tuning using normal-float 4 bit (nf4).  In practice, the quantization behind the scenes is handled by the `bitsandbytes` library.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face Base Model ID
model_id = "NousResearch/Llama-2-7b-hf" # non-gated

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=bnb_config, 
    use_cache=False, 
    device_map="auto")
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

With the model loaded up, we are ready to finetune using our dataset.

## Finetuning the Model

There are two main ways to finetune a large language model:

1. Pre-training/Full Finetuning

    In this situation, all of the model weights (all 7b of them) are set to be trainable and tweaked during training.  This can lead to the most dramatic changes in model behavior but is also the most computationally expensive.  
    
    When initially training the model, also known as pre-training, this is necessarily done and where you see the extreme computational costs show up (i.e. 500 A100 80GB GPUs trained for 10000 hours, etc...).

2. Parameter Efficient Fine-Tuning (PEFT)

    Parameter efficient finetuning methods are an alternative to full finetuning where, instead of training the parameters of the pre-trained model, a subset of new parameters are trained without touching the base model weights. These new trainable parameters are injected into the model mathematically at different points to change the outcome.  There are a handful of methods that use this approach such as Prompt Tuning, P-Tuning, and Low-Rank Adaptation (LoRA).  For this lab, we will focus on LoRA.  

    LoRA methods introduce a set of trainable rank-decomposition matrices (update matrices) which can be used to modify the existing weights of the pre-trained model.  The typical location that these matrices are placed are within the attention layers, so they are not exclusive to LLMs.  The size of these update matrices can be controlled by  setting the desired rank of the matrix (lora_r), with smaller rank corresponding to smaller matrices and thus fewer trainable parameters.   During fine-tuning, only these update matrices are tuned and often times, this makes the total number of trainable parameters a very small fraction of the total number of weights.

### Finetuning using `peft`

To configure the model for paremeter efficient fine-tuning and LoRA, we will use the `peft` package.  Specifically, we will define our Lora parameters and also set to the taks to `CAUSAL_LM` to train the model for generative purposes.  Because we also quantized the model to 4-bit, we will also be using a state-of-the-art method called Quantized LoRA (QLoRA) to do this training in low precision to save memory.


In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config for QLoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# prepare model for training with low-precision
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

### Running the trainer with `trl`

Now that we have prepared the data, loaded the model in 4-bit, and configured our LoRA finetuning according to our model, we are ready to train the model. Training of LLMs for generative purposes uses the causal language modeling objective.  Briefly, this specifies that when calculating attention, the model should only be able to consider things "to the left".  So for a sentence, it can only decide what to generate by looking at all of the words that came before it.  

A very useful wrapper for training transformer based models is the Supervised Fine-Tuning Trainer (`SFTrainer`) provided by the `trl` library.  While the supervised fine tuning is typically used in the context of reinforcement learning, for our purposes, it simply refers to providing the model with examples of input, and response.  All of the actual training, including computing gradients, tweaking the optimizer, batching the data, evaluation will be done behind the scenes using the `SFTrainer` wrapper.  This will conduct the finetuning that we want after we pass in the dataset and hyperparameters.  This is much more efficient and robust than writing our own training code.

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

args = TrainingArguments(
    output_dir="llama-7-int4-dolly", 
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)

# max sequence length for model and packing of the dataset
max_seq_length = 2048 

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)

With all of the configuration done, we can now run our training.  On an A10g, this takes about 3 hours to run, after which it will save the LoRA weights to the `llama-7-int4-dolly` directory.

In [ ]:
trainer.train() # there will not be a progress bar since tqdm is disabled

After the model has finished training, it is ready to be used.  Now, hopefully, when the model sees the prompt that we crafted before, it will know how to respond.

In [ ]:
trainer.save_model()